In [99]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [100]:
ex = pd.read_json('support_files/exercises-concepts.json')

In [101]:
total_exercise_count = ex['id'].count()
total_level_count = ex['total_levels'].sum()

In [102]:
df = pd.read_json('support_files/student_report_itk.json')

In [103]:
df.shape

(42347, 11)

In [104]:
df.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks
0,7695,Eva Admin,,,,13,Introduction to Gaming - Grade 5,1,3,0,"<xml xmlns=""https://developers.google.com/bloc..."
1,7695,Eva Admin,,,,13,Introduction to Gaming - Grade 5,2,3,0,
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc..."
3,1,,,100,A,2,Loopy Loops,1,12,0,
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc..."


In [105]:
df = df[~(df['grade']=='')]

In [106]:
df.shape

(41851, 11)

In [107]:
def count_blocks(xml):
    soup = BeautifulSoup(xml, 'html.parser')
    return len(soup.find_all('block'))

In [108]:
df['block_count'] = df['blocks'].apply(count_blocks)

In [109]:
def proper_case(string):
    parts = string.split(' ')

    return ' '.join(word.title() for word in parts[0:])

In [110]:
df['name'] = df['name'].apply(proper_case)

In [111]:
df.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",4
3,1,,,100,A,2,Loopy Loops,1,12,0,,0
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",6
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
6,1,,,100,A,1,Fun with Basics,4,10,0,,0


In [112]:
df.shape

(41851, 12)

In [113]:
df_lev_completed = df[df['level_completion_status']==1]

In [114]:
df_lev_completed.shape

(30909, 12)

# BLOCK 1 up 30% students

Levels Completed | School Average | Exercises Completed | School Average

In [115]:
df_ex_completed = df_lev_completed.groupby(['user_id', 'name', 'grade', 'division', 'exercise_id', 'total_levels']).agg(
    levels_done=('level_completion_status', 'sum')).reset_index()

In [116]:
df_ex_completed.head()

,user_id,name,grade,division,exercise_id,total_levels,levels_done
0,1,,100,A,1,10,3
1,4682,A Isha Angel,3,Garnet,1,10,5
2,4682,A Isha Angel,3,Garnet,2,12,1
3,4682,A Isha Angel,3,Garnet,8,8,1
4,4683,A Joy Samuel,2,Zircon,1,10,1


In [117]:
#to get students who have completed exercise more than 30% threshold and to add the ex_completion status flag

threshold = 0.3 * df_ex_completed['total_levels']
df_ex_completed['ex_completion_status'] = (df_ex_completed['levels_done']>threshold).astype(int)

In [118]:
df_ex_completed = df_ex_completed[['user_id', 'name', 'grade', 'division', 'exercise_id', 'ex_completion_status', 'total_levels',
       'levels_done']]

In [119]:
df_ex_completed.head()

,user_id,name,grade,division,exercise_id,ex_completion_status,total_levels,levels_done
0,1,,100,A,1,0,10,3
1,4682,A Isha Angel,3,Garnet,1,1,10,5
2,4682,A Isha Angel,3,Garnet,2,0,12,1
3,4682,A Isha Angel,3,Garnet,8,0,8,1
4,4683,A Joy Samuel,2,Zircon,1,0,10,1


In [120]:
#merging eitht the orignal df to get school for these students

df_wo_duplicates = df.drop_duplicates(subset=['user_id']).reset_index()

In [121]:
df_wo_duplicates.shape

(2023, 13)

In [122]:
report_block1 = pd.merge(df_ex_completed, df_wo_duplicates[['user_id', 'school']], on='user_id', how='left').reset_index()

In [123]:
report_block1.shape

(5113, 10)

In [124]:
report_block1.head()

,index,user_id,name,grade,division,exercise_id,ex_completion_status,total_levels,levels_done,school
0,0,1,,100,A,1,0,10,3,
1,1,4682,A Isha Angel,3,Garnet,1,1,10,5,Eva World School
2,2,4682,A Isha Angel,3,Garnet,2,0,12,1,Eva World School
3,3,4682,A Isha Angel,3,Garnet,8,0,8,1,Eva World School
4,4,4683,A Joy Samuel,2,Zircon,1,0,10,1,Eva World School


In [125]:
#removing all students who have not completed 30% of the ex based on the ex_completion_flag

report_block1 = report_block1[~(report_block1['ex_completion_status']==0)]

In [126]:
report_block1.shape

(3948, 10)

In [127]:
report_block1 = report_block1.groupby(['user_id', 'name', 'grade', 'division', 'school']).agg(total_ex_done=('ex_completion_status', 'sum'), total_levels_done=('levels_done', 'sum')).reset_index()

In [128]:
report_block1.head()

,user_id,name,grade,division,school,total_ex_done,total_levels_done
0,4682,A Isha Angel,3,Garnet,Eva World School,1,5
1,4685,Aadit Jignesh Limbani,1,Sapphire,Eva World School,4,29
2,4692,Aaradhya Harshal Sonar,4,Morgonite,Eva World School,1,5
3,4693,Aaradhya Singh,3,Garnet,Eva World School,2,12
4,4696,Aarav Amit Mathur,3,Garnet,Eva World School,1,4


In [129]:
report_block1.shape

(1765, 7)

In [130]:
#get the no of unique students in the df to further calculate the school average
#PS formulae is total no of (ex/lv) done / students who have done the same atlease once

unique_students = report_block1.groupby(['school']).agg(student_count = ('user_id', 'nunique'))

In [131]:
total_levels = report_block1.groupby(['school']).agg(lev_done = ('total_levels_done', 'sum'))

In [132]:
average_lev = pd.merge(unique_students, total_levels, on='school', how='left')

In [133]:
average_lev

,student_count,lev_done
school,,
Bhiwandi School,82,1457
DWPS,295,4762
Eva Indore,185,4241
Eva Thane,102,2180
Eva World School,501,7222
IRA,308,4083
SKS,292,4845


In [134]:
average_lev['average'] = (average_lev['lev_done'] / average_lev['student_count']).round(2)

In [135]:
average_lev_dict = dict(zip(list(average_lev.index), average_lev['average']))

In [136]:
average_lev_dict

{'Bhiwandi School': 17.77,
 'DWPS': 16.14,
 'Eva Indore': 22.92,
 'Eva Thane': 21.37,
 'Eva World School': 14.42,
 'IRA': 13.26,
 'SKS': 16.59}

In [137]:
total_ex = report_block1.groupby(['school']).agg(ex_done = ('total_ex_done', 'sum'))

In [138]:
total_ex

,ex_done
school,
Bhiwandi School,212
DWPS,627
Eva Indore,520
Eva Thane,287
Eva World School,1058
IRA,588
SKS,656


In [139]:
average_ex = pd.merge(unique_students, total_ex, on='school', how='left')

In [140]:
average_ex

,student_count,ex_done
school,,
Bhiwandi School,82,212
DWPS,295,627
Eva Indore,185,520
Eva Thane,102,287
Eva World School,501,1058
IRA,308,588
SKS,292,656


In [141]:
average_ex['average'] = (average_ex['ex_done'] / average_ex['student_count']).round(2)

In [142]:
average_ex_dict = dict(zip(list(average_ex.index), average_ex['average']))

In [143]:
average_ex_dict

{'Bhiwandi School': 2.59,
 'DWPS': 2.13,
 'Eva Indore': 2.81,
 'Eva Thane': 2.81,
 'Eva World School': 2.11,
 'IRA': 1.91,
 'SKS': 2.25}

In [144]:
report_block1['school_ex_avg'] = report_block1['school'].map(average_ex_dict)

In [145]:
report_block1['school_lev_avg'] = report_block1['school'].map(average_lev_dict)

In [146]:
report_block1.head()

,user_id,name,grade,division,school,total_ex_done,total_levels_done,school_ex_avg,school_lev_avg
0,4682,A Isha Angel,3,Garnet,Eva World School,1,5,2.11,14.42
1,4685,Aadit Jignesh Limbani,1,Sapphire,Eva World School,4,29,2.11,14.42
2,4692,Aaradhya Harshal Sonar,4,Morgonite,Eva World School,1,5,2.11,14.42
3,4693,Aaradhya Singh,3,Garnet,Eva World School,2,12,2.11,14.42
4,4696,Aarav Amit Mathur,3,Garnet,Eva World School,1,4,2.11,14.42


In [147]:
report_block1.shape

(1765, 9)

In [148]:
report_block1.to_csv('support_files/report_block1.csv')

# BLOCK 2

Exercise Name | Total Levels | Levels Completed | Concepts | Blocks Used

In [149]:
df_lev_completed.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count
2,1,,,100,A,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",4
4,1,,,100,A,1,Fun with Basics,2,10,1,"<xml xmlns=""https://developers.google.com/bloc...",6
5,1,,,100,A,1,Fun with Basics,3,10,1,"<xml xmlns=""https://developers.google.com/bloc...",9
110,7714,Aabha Patil,Bhiwandi School,1,Sapphire,1,Fun with Basics,1,10,1,"<xml xmlns=""https://developers.google.com/bloc...",3
111,7714,Aabha Patil,Bhiwandi School,1,Sapphire,45,Fun with Basics,1,3,1,"<xml xmlns=""https://developers.google.com/bloc...",10


In [150]:
report_block2 = df_lev_completed.groupby(['user_id', 'name', 'exercise_id', 'exercise_name', 'total_levels']).agg(levels_done=('level_completion_status', 'sum'), blocks_used=('block_count', 'sum')).reset_index()

In [151]:
threshold = 0.3 * df_ex_completed['total_levels']
report_block2['ex_completion_status'] = (report_block2['levels_done']>threshold).astype(int)

In [152]:
report_block2 = report_block2[~(report_block2['ex_completion_status']==0)]

In [153]:
report_block2.head()

,user_id,name,exercise_id,exercise_name,total_levels,levels_done,blocks_used,ex_completion_status
1,4682,A Isha Angel,1,Fun with Basics,10,5,43,1
8,4685,Aadit Jignesh Limbani,1,Fun with Basics,10,10,127,1
10,4685,Aadit Jignesh Limbani,45,Fun with Basics,3,3,41,1
11,4685,Aadit Jignesh Limbani,49,Fun with Basics - Grade 1 & 2,8,8,0,1
12,4685,Aadit Jignesh Limbani,50,Loopy Loops - Grade 1/2,8,8,0,1


In [154]:
report_block2[['exercise_name', 'total_levels']].value_counts()

exercise_name                  total_levels
Fun with Basics                10              1042
Fun with Basics - Grade 1 & 2  8                707
Loopy Loops                    12               572
Loopy Loops - Grade 1/2        8                567
Dog and the loops              8                371
Swamp conditionals             4                176
Conditional Crops              12               108
Fun with Basics                3                102
Baloon pop functions           8                 89
Functions on the field         9                 54
Loops and castles              8                 46
Loopy Loops                    4                 40
Predator bird functions        7                 38
Backyard Functions             10                27
                               6                  4
Gardening Conditionals         6                  3
Desert conditionals            4                  2
dtype: int64

# Block 3

Concepts | Explanation

In [155]:
ex.rename(columns={'id': 'exercise_id'}, inplace=True)

In [156]:
ex.head()

,exercise_id,name,ordern,description,image,total_levels,type,folder,video,disabled
0,1,Fun with Basics,1,"Sequence, Algorithmic Thinking",fun_with_basics.png,10,maze,fun_with_basics,FWB_LI FINAL.mp4,0
1,2,Loopy Loops,2,"Loops, Debugging",loopy_loops.png,12,maze,loopy_loops,,0
2,3,Conditional Crops,3,"Conditional Statements, Pattern Recognition",conditional_crops.png,12,maze,conditional_crops,,0
3,4,Backyard Functions,4,"Functions, Variables, Events",backyard_functions.png,10,maze,backyard_functions,,0
4,5,Dog and the loops,5,"Loops, Variables, Functions",Dog and the loops - Grade 4.png,8,maze,dog_and_the_loops,,0


In [157]:
report_block2 = pd.merge(report_block2, ex[['description', 'exercise_id']], on='exercise_id', how='left').reset_index()

In [158]:
columns = report_block2.columns.tolist()
columns

['index',
 'user_id',
 'name',
 'exercise_id',
 'exercise_name',
 'total_levels',
 'levels_done',
 'blocks_used',
 'ex_completion_status',
 'description']

In [160]:
report_block2 = report_block2[['user_id',
 'name',
 'exercise_id',
 'exercise_name',
 'description',
 'ex_completion_status',
 'levels_done',
 'total_levels',
 'blocks_used']]

In [161]:
report_block2.head()

,user_id,name,exercise_id,exercise_name,description,ex_completion_status,levels_done,total_levels,blocks_used
0,4682,A Isha Angel,1,Fun with Basics,"Sequence, Algorithmic Thinking",1,5,10,43
1,4685,Aadit Jignesh Limbani,1,Fun with Basics,"Sequence, Algorithmic Thinking",1,10,10,127
2,4685,Aadit Jignesh Limbani,45,Fun with Basics,"Sequence, Algorithmic Thinking",1,3,3,41
3,4685,Aadit Jignesh Limbani,49,Fun with Basics - Grade 1 & 2,"Sequence, Algorithmic Thinking",1,8,8,0
4,4685,Aadit Jignesh Limbani,50,Loopy Loops - Grade 1/2,"Loops, Debugging",1,8,8,0


In [162]:
report_block2.to_csv('support_files/report_block2.csv')

# BLOCK 1 down 30% students

Levels Completed | School Average | Exercises Completed | School Average

In [223]:
df_lev_not_completed = df[df['level_completion_status']==0]

In [224]:
df_lev_not_completed.shape

(10942, 12)

In [225]:
df_lev_not_completed['flag'] = df_lev_not_completed['user_id'].isin(df_lev_completed['user_id']).astype(int)

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/3516208275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lev_not_completed['flag'] = df_lev_not_completed['user_id'].isin(df_lev_completed['user_id']).astype(int)


In [226]:
report_block1_down30 = df_lev_not_completed[df_lev_not_completed['flag']==0]

In [227]:
report_block1.head()

,user_id,name,grade,division,school,total_ex_done,total_levels_done,school_ex_avg,school_lev_avg
0,4682,A Isha Angel,3,Garnet,Eva World School,1,5,2.11,14.42
1,4685,Aadit Jignesh Limbani,1,Sapphire,Eva World School,4,29,2.11,14.42
2,4692,Aaradhya Harshal Sonar,4,Morgonite,Eva World School,1,5,2.11,14.42
3,4693,Aaradhya Singh,3,Garnet,Eva World School,2,12,2.11,14.42
4,4696,Aarav Amit Mathur,3,Garnet,Eva World School,1,4,2.11,14.42


In [228]:
report_block1_down30.head()

,user_id,name,school,grade,division,exercise_id,exercise_name,level,total_levels,level_completion_status,blocks,block_count,flag
3059,6055,Anaika Kaur Sawhney,DWPS,2,Opal,49,Fun with Basics - Grade 1 & 2,1,8,0,,0,0
5790,7688,Ankush Swapan Samanta,DWPS,5,ADAMITE,3,Conditional Crops,1,12,0,"<xml xmlns=""https://developers.google.com/bloc...",24,0
5791,7688,Ankush Swapan Samanta,DWPS,5,ADAMITE,5,Dog and the loops,1,8,0,"<xml xmlns=""https://developers.google.com/bloc...",6,0
5792,7688,Ankush Swapan Samanta,DWPS,5,ADAMITE,13,Introduction to Gaming - Grade 5,1,3,0,"<xml xmlns=""https://developers.google.com/bloc...",2,0
8046,6040,Adee Zoheb Patankar,DWPS,7,Amethyst,10,Desert conditionals,1,4,0,"<xml xmlns=""https://developers.google.com/bloc...",5,0


In [229]:
report_block1_down30.drop(['exercise_id', 'exercise_name', 'level', 'total_levels', 'blocks', 'block_count', 'flag'], axis=1, inplace=True)

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/224180183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block1_down30.drop(['exercise_id', 'exercise_name', 'level', 'total_levels', 'blocks', 'block_count', 'flag'], axis=1, inplace=True)


In [230]:
additional_columns = ['total_ex_done', 'total_levels_done']

for column in additional_columns:
    report_block1_down30[column] = 0

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/3615563633.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block1_down30[column] = 0
/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/3615563633.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block1_down30[column] = 0


In [231]:
report_block1_down30 = report_block1_down30[['user_id', 'name', 'grade', 'division', 'school','total_ex_done','total_levels_done']]

In [232]:
report_block1_down30['school_ex_avg'] = report_block1_down30['school'].map(average_ex_dict)
report_block1_down30['school_lev_avg'] = report_block1_down30['school'].map(average_lev_dict)


In [234]:
report_block1_down30.to_csv('./support_files/report_block1_down30.csv')

In [ ]:
exit()

# BLOCK 2 down 30% report

In [235]:
report_block2_down30 = report_block1_down30.groupby(['user_id', 'name']).agg(user_id_count = ('user_id', 'nunique')).reset_index()

In [236]:
report_block2_down30.drop('user_id_count', axis=1, inplace=True)

In [237]:
addtional_report_block2_down30_1 = ['exercise_id', 'exercise_name', 'description']

for column in addtional_report_block2_down30_1:
    report_block2_down30[column] = 'None'

addtional_report_block2_down30_2 = ['ex_completion_status', 'levels_done', 'total_levels', 'blocks_used']
for column in addtional_report_block2_down30_2:
    report_block2_down30[column] = 0

In [238]:
report_block2_down30.head()

,user_id,name,exercise_id,exercise_name,description,ex_completion_status,levels_done,total_levels,blocks_used
0,4695,Aaradhya Vijay Kamble,None,None,None,0,0,0,0
1,4759,Darsh Sharma,None,None,None,0,0,0,0
2,4765,Dhruv Atul Bangal,None,None,None,0,0,0,0
3,4807,Kruti Chaman Pandey,None,None,None,0,0,0,0
4,4839,Nidhi Pravin More,None,None,None,0,0,0,0


In [239]:
report_block2_down30.to_csv('./support_files/report_block2_down30.csv')

# No Exercise Done Report

In [242]:
df_na = pd.read_json('support_files/student_report_itk_na.json')

In [243]:
df_na = df_na[~(df_na['grade']=='')]

In [244]:
df_na['name'] = df_na['name'].apply(proper_case)

In [245]:
df_na.head()

,user_id,name,school,grade,division
3,7742,Reyansh Ojha,Bhiwandi School,2,Zircon
4,8004,Adi Hariya,Bhiwandi School,6,Turquoise
5,8003,Ayush Choudhary,Bhiwandi School,7,Amethyst
6,6138,Kshitij Ravi Divakar,DWPS,1,Sapphire
7,6178,Navika Naveen Poojary,DWPS,1,Sapphire


In [261]:
report_block1_na = df_na

In [262]:
additional_columns = ['total_ex_done', 'total_levels_done']

for column in additional_columns:
    report_block1_na[column] = 0

In [263]:
report_block1_na = report_block1_na[['user_id', 'name', 'grade', 'division', 'school', 'total_ex_done',
       'total_levels_done']]

In [264]:
report_block1_na['school_ex_avg'] = report_block1_na['school'].map(average_ex_dict)

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/3270452955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block1_na['school_ex_avg'] = report_block1_na['school'].map(average_ex_dict)


In [265]:
report_block1_na['school_lev_avg'] = report_block1_na['school'].map(average_lev_dict)

In [266]:
report_block1_na.head()

,user_id,name,grade,division,school,total_ex_done,total_levels_done,school_ex_avg,school_lev_avg
3,7742,Reyansh Ojha,2,Zircon,Bhiwandi School,0,0,2.59,17.77
4,8004,Adi Hariya,6,Turquoise,Bhiwandi School,0,0,2.59,17.77
5,8003,Ayush Choudhary,7,Amethyst,Bhiwandi School,0,0,2.59,17.77
6,6138,Kshitij Ravi Divakar,1,Sapphire,DWPS,0,0,2.13,16.14
7,6178,Navika Naveen Poojary,1,Sapphire,DWPS,0,0,2.13,16.14


In [268]:
report_block1_na.to_csv('support_files/report_block1_na.csv')

In [275]:
report_block2_na = df_na[['user_id', 'name']]

In [276]:
report_block2_na.head()

,user_id,name
3,7742,Reyansh Ojha
4,8004,Adi Hariya
5,8003,Ayush Choudhary
6,6138,Kshitij Ravi Divakar
7,6178,Navika Naveen Poojary


In [277]:
addtional_columns_block2_1 = ['exercise_id', 'exercise_name', 'description']

for column in addtional_columns_block2_1:
    report_block2_na[column] = 'None'

addtional_columns_block2_2 = ['ex_completion_status', 'levels_done', 'total_levels', 'blocks_used']
for column in addtional_columns_block2_2:
    report_block2_na[column] = 0

/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/2739211612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block2_na[column] = 'None'
/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/2739211612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_block2_na[column] = 'None'
/var/folders/2f/qrm1hg3945j9tqtn7tmw3vq40000gn/T/ipykernel_1209/2739211612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [281]:
report_block2_na.head()

,user_id,name,exercise_id,exercise_name,description,ex_completion_status,levels_done,total_levels,blocks_used
3,7742,Reyansh Ojha,None,None,None,0,0,0,0
4,8004,Adi Hariya,None,None,None,0,0,0,0
5,8003,Ayush Choudhary,None,None,None,0,0,0,0
6,6138,Kshitij Ravi Divakar,None,None,None,0,0,0,0
7,6178,Navika Naveen Poojary,None,None,None,0,0,0,0


In [283]:
report_block2_na.to_csv('support_files/report_block2_na.csv')

: 